In [1]:
import os #to read and write a file
import cv2 #opencv for manipulating images
from glob import glob #finds all the pathnames matching a specified pattern
import h5py #will be helpful for storing huge amt of numerical data
import shutil #offers a high-level operations on files and collection of files
import imgaug as aug #Image augmentation. Helful for creating much more larger dataset from our input.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns #visualising tool
import matplotlib.pyplot as plt #visualising tool
import imgaug.augmenters as iaa #Image augmentation.

from os import listdir, makedirs, getcwd, remove 
#listdr- return list containing names of the entries given in path
#make directory named path with the specified numeric mode.
#getcwd - getting current working directory
#remove- remove/delete a file path

from os.path import isfile, join, abspath, exists, isdir
#isfile - to check specified path is available in that file or not.
#join - to join one or more path.
#abspath - returns a normalised version of the path
#isdir- returns true/false if specified path is there in the directory or not.

from pathlib import Path #object oriented file system path.
from skimage.io import imread #image reading/writing
from skimage.transform import resize #resize
from keras.models import Sequential, Model, load_model #keras NN model
from keras.applications.vgg16 import VGG16, preprocess_input #VGG16
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten #layers to build NN
from keras.optimizers import Adam, SGD, RMSprop #optimizers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split 

import tensorflow as tf

Using TensorFlow backend.


In [2]:
columns1 = ['path', 'label']
yes_xray_df = pd.DataFrame(columns = columns1)
all_imgs_path_yes = {os.path.basename(x): x for x in glob(os.path.join('brain_tumor_dataset' ,'yes', '*.jpg'))}
print('Scans found:', len(all_imgs_path_yes))

Scans found: 255


In [3]:
#labeling 1, they have tumor
yes_xray_df['path'] = all_imgs_path_yes.values()
yes_list = [1]*255
yes_xray_df['label'] = yes_list 

In [4]:
#labeling 0, they dont have tumor
columns = ['path', 'label']
no_xray_df = pd.DataFrame(columns = columns)
no_imgs_path = {os.path.basename(x): x for x in glob(os.path.join('brain_tumor_dataset' ,'no', '*.jpg'))}

print('Scans found:', len(no_imgs_path))

no_xray_df['path'] = no_imgs_path.values()

no_list = [0]*255 #Labeling 0 for all 255 records
no_xray_df['label'] = no_list

Scans found: 255


In [5]:
yes_xray_df.head()

,path,label
0,brain_tumor_dataset\yes\yes1.jpg,1
1,brain_tumor_dataset\yes\yes10.jpg,1
2,brain_tumor_dataset\yes\yes100.jpg,1
3,brain_tumor_dataset\yes\yes101.jpg,1
4,brain_tumor_dataset\yes\yes102.jpg,1


In [6]:
#pred data

columns2 = ['path', 'label']
pred_xray_df = pd.DataFrame(columns = columns)
all_imgs_path_pred = {os.path.basename(x): x for x in glob(os.path.join('brain_tumor_dataset' ,'pred','*.jpg'))}

print('Scans found:', len(all_imgs_path_pred))

pred_xray_df['path'] = all_imgs_path_pred.values()
pred_xray_df.head() # there are no Labels for this dataset.

Scans found: 60


,path,label
0,brain_tumor_dataset\pred\pred1.jpg,NaN
1,brain_tumor_dataset\pred\pred10.jpg,NaN
2,brain_tumor_dataset\pred\pred11.jpg,NaN
3,brain_tumor_dataset\pred\pred12.jpg,NaN
4,brain_tumor_dataset\pred\pred13.jpg,NaN


In [7]:
frames = [yes_xray_df, no_xray_df]

In [8]:
final_df = pd.concat(frames)
final_df = final_df.sample(frac=1.).reset_index(drop = True) #shuffling the rows
print("Shuffling the dataset")
final_df.head(5)

Shuffling the dataset


,path,label
0,brain_tumor_dataset\no\no144.jpg,0
1,brain_tumor_dataset\no\no179.jpg,0
2,brain_tumor_dataset\no\no72.jpg,0
3,brain_tumor_dataset\yes\yes152.jpg,1
4,brain_tumor_dataset\yes\yes126.jpg,1


In [9]:
#train_test split
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(final_df, test_size = 0.25, random_state = 2020)
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

train 382 validation 128


In [10]:
train_df

,path,label
434,brain_tumor_dataset\yes\yes184.jpg,1
415,brain_tumor_dataset\no\no147.jpg,0
341,brain_tumor_dataset\yes\yes4.jpg,1
103,brain_tumor_dataset\no\no157.jpg,0
357,brain_tumor_dataset\no\no46.jpg,0
...,...,...
195,brain_tumor_dataset\yes\yes176.jpg,1
118,brain_tumor_dataset\yes\yes90.jpg,1
323,brain_tumor_dataset\no\no49.jpg,0
392,brain_tumor_dataset\no\no255.jpg,0


In [11]:
valid_df

,path,label
297,brain_tumor_dataset\yes\yes162.jpg,1
326,brain_tumor_dataset\no\no131.jpg,0
229,brain_tumor_dataset\yes\yes235.jpg,1
424,brain_tumor_dataset\yes\yes156.jpg,1
476,brain_tumor_dataset\no\no207.jpg,0
...,...,...
145,brain_tumor_dataset\no\no28.jpg,0
155,brain_tumor_dataset\yes\yes236.jpg,1
503,brain_tumor_dataset\no\no156.jpg,0
100,brain_tumor_dataset\no\no234.jpg,0


In [12]:
# dimensions to consider for the images
img_rows, img_cols, img_channels = 224,224,3

# batch size for training  
batch_size=8

# total number of classes in the dataset
nb_classes=2

In [13]:
#augmentation
seq = iaa.OneOf([
    iaa.Fliplr(), 
    iaa.Affine(rotate=20), 
    iaa.Multiply((1.2, 1.5))]) 

#Fliplr- Horizontal Flips
#Affine - rotation
#Multiply - Random Brightness

In [14]:
def data_generator(data, batch_size, is_validation_data=False):
    # Get total number of samples in the data
    n = len(data)
    nb_batches = int(np.ceil(n/batch_size))

    # Get a numpy array of all the indices of the input data
    indices = np.arange(n)
    
    # Define two numpy arrays for containing batch data and labels
    batch_data = np.zeros((batch_size, img_rows, img_cols, img_channels), dtype=np.float32)
    batch_labels = np.zeros((batch_size, nb_classes), dtype=np.float32)
    
    while True:
        if not is_validation_data:
            # shuffle indices for the training data
            np.random.shuffle(indices)
            
        for i in range(nb_batches):
            # get the next batch 
            next_batch_indices = indices[i*batch_size:(i+1)*batch_size]
            
            # process the next batch
            for j, idx in enumerate(next_batch_indices):
                img = cv2.imread(data.iloc[idx]["path"])
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                label = data.iloc[idx]["label"]
                
                if not is_validation_data:
                    img = seq.augment_image(img)
                
                img = cv2.resize(img, (img_rows, img_cols)).astype(np.float32)
                batch_data[j] = img
                batch_labels[j] = to_categorical(label,num_classes=nb_classes)
            
            batch_data = preprocess_input(batch_data)
            yield batch_data, batch_labels

In [15]:
#training data generator 
train_data_gen = data_generator(train_df, batch_size)

# validation data generator 
valid_data_gen = data_generator(valid_df, batch_size, is_validation_data=True)

In [16]:
#Modeling part
#Transfer Learning
#Choosing VGG16

def get_base_model():
    base_model = VGG16(input_shape=(img_rows, img_cols, img_channels), weights='imagenet', include_top=True)
    return base_model

In [17]:
# get the base model
base_model = get_base_model()

#  get the output of the second last dense layer 
base_model_output = base_model.layers[-2].output

# add new layers 
x = Dropout(0.5,name='drop2')(base_model_output)
output = Dense(2, activation='softmax', name='fc3')(x)

In [18]:
# define a new model 
model = Model(base_model.input, output)

# Freeze all the base model layers 
for layer in base_model.layers[:-1]:
    layer.trainable=False

In [19]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(0.001), metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [20]:
# the restore_best_weights parameter load the weights of the best iteration once the training finishes
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10, restore_best_weights=True)

# checkpoint to save model
chkpt = ModelCheckpoint(filepath="model1", save_best_only=True)

# number of training and validation steps for training and validation
nb_train_steps = int(np.ceil(len(train_df)/batch_size))
nb_valid_steps = int(np.ceil(len(valid_df)/batch_size))

# number of epochs 
nb_epochs=30

In [21]:
nb_train_steps

48

In [ ]:
history1 = model.fit_generator(train_data_gen, 
                              epochs=nb_epochs, 
                              steps_per_epoch=nb_train_steps, 
                              validation_data=valid_data_gen, 
                              validation_steps=nb_valid_steps)

Epoch 1/30
48/48 [==============================] - 121s 3s/step - loss: 1.2944 - accuracy: 0.6068 - val_loss: 2.1406 - val_accuracy: 0.6094
Epoch 2/30
48/48 [==============================] - 127s 3s/step - loss: 0.9029 - accuracy: 0.7292 - val_loss: 1.7938 - val_accuracy: 0.7266
Epoch 3/30
48/48 [==============================] - 130s 3s/step - loss: 0.7317 - accuracy: 0.7682 - val_loss: 0.7509 - val_accuracy: 0.7656
Epoch 4/30
48/48 [==============================] - 127s 3s/step - loss: 0.7778 - accuracy: 0.7760 - val_loss: 1.4002 - val_accuracy: 0.7812
Epoch 5/30
48/48 [==============================] - 165s 3s/step - loss: 0.7744 - accuracy: 0.7812 - val_loss: 1.6101 - val_accuracy: 0.7734
Epoch 6/30
48/48 [==============================] - 177s 4s/step - loss: 0.6315 - accuracy: 0.7891 - val_loss: 1.3140 - val_accuracy: 0.7891
Epoch 7/30
48/48 [==============================] - 170s 4s/step - loss: 0.7567 - accuracy: 0.8073 - val_loss: 2.7772 - val_accuracy: 0.7578
Epoch 8/30
48

In [ ]:
#get the training and validation accuracy
train_acc = history1.history['accuracy']
valid_acc = history1.history['val_accuracy']

#get the loss
train_loss = history1.history['loss']
valid_loss = history1.history['val_loss']

#get the entries
xvalues = np.arange(len(train_acc))

#visualise
f, ax = plt.subplots(1,2, figsize = (10,5))
ax[0].plot(xvalues, train_loss)
ax[0].plot(xvalues, valid_loss)
ax[0].set_title("Loss curve")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("loss")
ax[0].legend(['train', 'validation'])

ax[1].plot(xvalues, train_acc)
ax[1].plot(xvalues, valid_acc)
ax[1].set_title("Accuracy")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("accuracy")
ax[1].legend(['train', 'validation'])

plt.show()

In [ ]:
valid_loss, valid_acc = model.evaluate_generator(valid_data_gen, steps=nb_valid_steps)
print(f"Final validation accuracy: {valid_acc*100:.2f}%")